# Pixivから検索結果をダウンロード

### ログインのために必要な情報
- refresh token

### メモ
ログイン時に、Cloudflareにブロックされたら時間をおいて、ログイン部分を再実行する

In [ ]:
from pixivpy3 import *
from time import sleep
import os
import datetime

In [ ]:
#@title 検索したいワード
search_word = 'NovelAI' #@param {type:"string"}

In [ ]:
#@title Parameters
TOKEN = "your_refresh_token" #@param {type:"string}
DL_PATH = "./download" #@param {type:"string"}
METADATA_PATH = "./metadata" #@param {type:"string"}
INTERVAL = 1 #@param {type:"int"}
DL_SIZE = "large" #@param ["square_medium", "medium", "large"]
START_DATE = datetime.date(2022, 8, 1) #@param {type:"date"}
END_DATE = datetime.date.today() #@param {type:"date"}
SEARCH_TARGET = "partial_match_for_tags" #@param ["exact_match_for_tags", "partial_match_for_tags", "title_and_caption", "keyword"]

In [ ]:
#@title Login
api = AppPixivAPI()
api.auth(refresh_token=TOKEN)

In [ ]:
#@title Download
if not os.path.exists(DL_PATH):
    os.makedirs(DL_PATH)
if not os.path.exists(METADATA_PATH):
    os.makedirs(METADATA_PATH)

for d in range((END_DATE - START_DATE).days + 1):
    date = START_DATE + datetime.timedelta(days=d)
    date_str = date.strftime("%Y-%m-%d")
    print("Downloading images in %s" % date_str)
    json_result = api.search_illust(search_word, start_date=date_str, end_date=date_str, search_target=SEARCH_TARGET)
    if not json_result.illusts:
        print("No images found in %s" % date_str)
        sleep(INTERVAL)
        continue
    date_dl_path = DL_PATH + "/" + date_str
    if not os.path.exists(date_dl_path):
        os.makedirs(date_dl_path)

    while True:
        for illust in json_result.illusts:
            print("[%s] %s" % (illust.title, illust.image_urls[DL_SIZE]))
            api.download(illust.image_urls[DL_SIZE], path=date_dl_path)
            with open(METADATA_PATH + "/" + date_str + ".txt", "a", encoding="utf-8") as f:
                f.write(str(illust) + "\n")
            sleep(INTERVAL)
        next_qs = api.parse_qs(json_result.next_url)
        with open(METADATA_PATH + "/next_qs.txt", "a", encoding="utf-8") as f:
            f.write(str(next_qs) + "\n")
            f.write(str(json_result.next_url) + "\n")
        if next_qs:
            json_result = api.search_illust(**next_qs)

            # Retry
            # *Note: authentification is required for every 3600 seconds
            for _ in range(10):
                if json_result.illusts:
                    break
                try:
                    api.auth(refresh_token=TOKEN)
                    json_result = api.search_illust(**next_qs)
                    sleep(INTERVAL*10)
                except:
                    pass
            if not json_result.illusts:
                with open(METADATA_PATH + "/error.txt", "a", encoding="utf-8") as f:
                    f.write(str(next_qs) + "\n")
                    f.write(str(json_result) + "\n")
                break
        else:
            break
        sleep(INTERVAL)